In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import polars as pl
import os
import random

from plotly import express as px

In [2]:
train, test = pd.read_parquet(os.getcwd() + "\\data\\train.pqt"), pd.read_parquet(os.getcwd() + "\\data\\test.pqt")
train.columns, test.columns = ["userid", "productid", "timestamp", "type"], ["userid", "productid", "timestamp", "type"]
train, test = train.sort_values(['userid', "timestamp"]).reset_index(drop=True), test.sort_values(['userid', "timestamp"]).reset_index(drop=True)

timeout = int(5 * 60 * 1e6)  # 5 min timer for session length
train["new_session"] = ((train.timestamp.diff() > timeout) | (train.userid.diff() != 0)).astype("int")
train["session"] = train.groupby("userid").new_session.cumsum() - 1
train = train.drop("new_session", axis=1)
train["usersession"] = train.userid.astype(str) + "_" + train.session.astype(str)

test["new_session"] = ((test.timestamp.diff() > timeout) | (test.userid.diff() != 0)).astype("int")
test["session"] = test.groupby("userid").new_session.cumsum() - 1
test = test.drop("new_session", axis=1)
test["usersession"] = test.userid.astype(str) + "_" + test.session.astype(str)

In [ ]:
train.head()

In [ ]:
test.head()

# Histograms

## Events

In [ ]:
px.bar(train.type.value_counts())

In [ ]:
px.bar(test.type.value_counts())

## User ids

In [ ]:
px.scatter(train.userid.value_counts().sort_values(ascending=False).reset_index(drop=True)[:10000])

# Product ids

In [ ]:
px.scatter(train.productid.value_counts().sort_values(ascending=False).reset_index(drop=True)[:10000])

# Sub Sampling

In [ ]:
sample_size = train.userid.unique().shape[0] / 100
sample_sessions = random.sample(list(train.userid.unique()), int(sample_size))
trains = train[train.userid.isin(sample_sessions)]
trains

In [10]:
trains.userid.value_counts()

121937    484
19203     435
14644     420
50063     419
167754    404
         ... 
88827       2
175531      2
88632       2
175965      2
174235      2
Name: userid, Length: 2000, dtype: int64

# Product Association

In [ ]:
visit_matrix = trains.groupby("usersession").productid.value_counts().unstack().fillna(0).T
visit_matrix

In [12]:
# covisit_matrix = visit_matrix.dot(visit_matrix.T)
# covisit_matrix